<a href="https://colab.research.google.com/github/nemat-al/Omdena_KitweChapter_LocalNewsAggregatorAnalyzer/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Trying to push to dagshub
```
# Installation of DagsHub
%pip install -q dagshub
# Cloning
!git clone https://dagshub.com/Omdena/KitweChapter_LocalNewsAggregatorAnalyzer.git
# Moving to the respository folder
%cd /content/KitweChapter_LocalNewsAggregatorAnalyzer
!git add .
!git commit -m "Data Preprocseeing"
!git push origin main
```



### Libraries

In [13]:
import pandas as pd
import csv
import string
import nltk
from nltk.corpus import wordnet, words
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [14]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

### Reading Dataset

Treating the Google Sheet similar to a CSV file and load it directly into a Pandas DataFrame using the Google Sheets link and a bit of URL manipulation (changing the `edit?usp=sharing` to `edit?usp=sharing`)

In [15]:
import pandas as pd

# Google Sheets URL (Modified for CSV export)
url = 'https://docs.google.com/spreadsheets/d/10bnfKtE32i15SqDKXtgJcBQdhH8aVuV5QBe3s6u1inY/export?format=csv'

# Load the data into a pandas DataFrame
data = pd.read_csv(url)

# Display the first few rows of the data
data.head()


,Source,Category,Headline,Link,Description,Date,Author
0,Daily Mail Zambia,"Headlines, News",K8 million CBU hall building starts,https://www.daily-mail.co.zm/2024/10/15/k8-mil...,MWILA NTAMBI Kitwe COPPERBELT University (CBU)...,"Tue, 15 Oct 2024 06:41:52 +0000",Website Editor
1,Daily Mail Zambia,"Headlines, News","Exercise aimed at recruiting 300,000 members i...",https://www.daily-mail.co.zm/2024/10/15/exerci...,MELODY MUPETA Kitwe THE United Party for Natio...,"Tue, 15 Oct 2024 06:17:07 +0000",Website Editor
2,Daily Mail Zambia,"Headlines, News",K8 million CBU hall building starts,https://www.daily-mail.co.zm/2024/10/15/k8-mil...,MWILA NTAMBI Kitwe COPPERBELT University (CBU)...,"Tue, 15 Oct 2024 06:41:52 +0000",Website Editor
3,Daily Mail Zambia,"Headlines, News","Exercise aimed at recruiting 300,000 members i...",https://www.daily-mail.co.zm/2024/10/15/exerci...,MELODY MUPETA Kitwe THE United Party for Natio...,"Tue, 15 Oct 2024 06:17:07 +0000",Website Editor
4,Daily Mail Zambia,"Headlines, News",K8 million CBU hall building starts,https://www.daily-mail.co.zm/2024/10/15/k8-mil...,MWILA NTAMBI Kitwe COPPERBELT University (CBU)...,"Tue, 15 Oct 2024 06:41:52 +0000",Website Editor


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14344 entries, 0 to 14343
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Source       14344 non-null  object
 1   Category     13870 non-null  object
 2   Headline     14343 non-null  object
 3   Link         14344 non-null  object
 4   Description  14330 non-null  object
 5   Date         14344 non-null  object
 6   Author       14343 non-null  object
dtypes: object(7)
memory usage: 784.6+ KB


### 1. Check Data Quality

#### 1.1 Checking for duplicates

In [17]:
# Drop the duplicated rows
data.drop_duplicates(inplace=True)

In [18]:
data.shape

(12346, 7)

#### 1.2 Check Data Quality

In [19]:
# Check for missing values
missing_values = data.isnull().sum()

In [20]:
# what rows has null values?
missing_values

,0
Source,0
Category,474
Headline,1
Link,0
Description,14
Date,0
Author,1


Among the null values, let's check the null `Headline` and `Description`

In [21]:
# Checking rows that have null description, I wanna check some of the links whether any data is inside them or not
missing_rows = data[data['Description'].isna()]

In [22]:
missing_rows

,Source,Category,Headline,Link,Description,Date,Author
4100,Lusaka Times,Photo Gallery,2018 COSAFA U20 Championship in Pictures,https://www.lusakatimes.com/2018/12/15/2018-co...,NaN,"Sat, 15 Dec 2018 05:07:43 +0000",Chief Editor
4371,Lusaka Times,Videos and Audios,President Lungu’s Arrival in Kitwe,https://www.lusakatimes.com/2018/06/07/preside...,NaN,"Thu, 07 Jun 2018 04:45:30 +0000",Chief Editor
4849,Lusaka Times,Photo Gallery,The Week in Pictures,https://www.lusakatimes.com/2017/05/05/the-wee...,NaN,"Fri, 05 May 2017 11:04:35 +0000",editor
4967,Lusaka Times,Photo Gallery,President and Vice President Visits to Army Wo...,https://www.lusakatimes.com/2017/01/06/preside...,NaN,"Fri, 06 Jan 2017 14:41:41 +0000",editor
5043,Lusaka Times,Photo Gallery,This Week in Pictures,https://www.lusakatimes.com/2016/10/14/this-we...,NaN,"Fri, 14 Oct 2016 13:24:10 +0000",editor
5243,Lusaka Times,Photo Gallery,President Lungu’s Kitwe Visit in Pictures,https://www.lusakatimes.com/2016/05/06/preside...,NaN,"Fri, 06 May 2016 10:58:38 +0000",editor
5945,Lusaka Times,Photo Gallery,UPND Kitwe Rally in Pictures,https://www.lusakatimes.com/2014/12/20/upnd-ki...,NaN,"Sat, 20 Dec 2014 20:10:29 +0000",editor
8549,Kitwe Online,"Theatre, auditions, Kitwe, little theatre, muk...",Kitwe Little Theatre Auditions – June 2022,https://kitweonline.com/kitweonline/discover-k...,NaN,"Thu, 16 Jun 2022 07:26:13 +0000",JS
8561,Kitwe Online,"Art, Kitwe On Line, Self-Development, sotambe ...",SOTAMBE FILM MAKING BOOTCAMP,https://kitweonline.com/kitweonline/sotambe-fi...,NaN,"Thu, 18 Feb 2021 13:05:02 +0000",JS
8562,Kitwe Online,"History, Kitwe On Line, Kitwe, town, video",Kitwe Town Video,https://kitweonline.com/kitweonline/kitwe-town...,NaN,"Tue, 02 Feb 2021 12:53:37 +0000",JS


In [23]:
# take a closer look at a row with a null description
print(data.loc[11044])

Source                                              Lusaka Voice
Category                                            GENERAL NEWS
Headline                              PF nominations in pictures
Link           https://lusakavoice.com/2016/06/01/pf-nominati...
Description                                                  NaN
Date                             Wed, 01 Jun 2016 11:04:40 +0000
Author                                             Online Editor
Name: 11044, dtype: object


Checked some links, some of them are empty like the one with `8728` id and [link](https://kitweonline.com/kitweonline/business/c) and some are link for photos [link](https://lusakavoice.com/2016/06/01/pf-nominations-in-pictures/), so guess best thing to do delete those rows with no description

In [24]:
# Drop rows with missing values in important columns
data = data.dropna(subset=['Description'])

In [25]:
data.shape

(12332, 7)

In [26]:
data[data['Headline'].isna()]

,Source,Category,Headline,Link,Description,Date,Author
10670,Daily Nations Zambia,NaN,NaN,https://dailynationzambia.com/2021/03/10304/?u...,"Mon, 30 Nov -0001 00:00:00 +0000 WARRIORS HOLD...","Sun, 07 Mar 2021 10:31:11 +0000",Daily Nation


In [27]:
# Checking the row with null headline
print(data.loc[10670])

Source                                      Daily Nations Zambia
Category                                                     NaN
Headline                                                     NaN
Link           https://dailynationzambia.com/2021/03/10304/?u...
Description    Mon, 30 Nov -0001 00:00:00 +0000 WARRIORS HOLD...
Date                             Sun, 07 Mar 2021 10:31:11 +0000
Author                                              Daily Nation
Name: 10670, dtype: object


Checked the link, it is legit, so mybe let's keep it for now


### 2. Text Preprocessing

let's take a look at one of the contents

In [28]:
print(data.loc[0]['Description'])

MWILA NTAMBI Kitwe COPPERBELT University (CBU) in Kitwe has launched construction of a 1,000-seater auditorium at K8 million. CBU vice-chancellor Imasiku Nyambe says building of the auditorium is a milestone as it will enhance infrastructure and the learning environment at the higher learning institution. He said once finished, the auditorium will host academic conferences and [&#8230;]


The sequence `&#8230`; is an HTML entity that represents an ellipsis (…), which is a punctuation mark consisting of three dots. It is commonly used to indicate that content has been truncated or is continuing.
In the context of RSS feeds:
When you see &#8230; at the end of a news content snippet, it usually means that the article content has been shortened or truncated in the feed.

In [29]:
def process_text(data, text_column='Description'):
    # Initialize the lemmatizer and stop words set
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # Define a function for processing each text entry
    def clean_text(text):
        # Remove URLs
        text = re.sub(r"http\S+", "", text)
        # Remove ellipses (both … and &#8230;)
        text = re.sub(r'…|&#8230;', '', text)
        # Tokenize, lower case, remove stopwords, punctuation, and lemmatize
        tokens = [
            lemmatizer.lemmatize(word.lower())
            for word in word_tokenize(text)
            if word.lower() not in stop_words and word not in string.punctuation
        ]
        return ' '.join(tokens)

    # Apply the cleaning function to the specified text column
    data['tokens'] = (data[text_column].apply(clean_text))

    # Optionally, you can create a cleaned text version as well
    data['cleaned_text'] = data[text_column].apply(lambda x: re.sub(r"http\S+", "", x))

    return data


In [30]:
data = process_text(data)

In [31]:
data['tokens'].loc[0]

'mwila ntambi kitwe copperbelt university cbu kitwe launched construction 1,000-seater auditorium k8 million cbu vice-chancellor imasiku nyambe say building auditorium milestone enhance infrastructure learning environment higher learning institution said finished auditorium host academic conference'

In [32]:
data['Description'].loc[0]

'MWILA NTAMBI Kitwe COPPERBELT University (CBU) in Kitwe has launched construction of a 1,000-seater auditorium at K8 million. CBU vice-chancellor Imasiku Nyambe says building of the auditorium is a milestone as it will enhance infrastructure and the learning environment at the higher learning institution. He said once finished, the auditorium will host academic conferences and [&#8230;]'

In [33]:
 # Save the cleaned dataframe
data.to_csv('/content/drive/MyDrive/Colab Notebooks/Omdena Kitwe News Aggregator/cleaned_news_data.csv', index=False)